In [1]:
#2018/5/15改
import requests
import pandas
from bs4 import BeautifulSoup

In [2]:
website = 'https://www.ptt.cc/cls/1'
global board_path,over18,nPage
board_path = list() #看板的路徑
over18 = dict({'yes':'yes'}) #處理18禁 之後都直接post不get
nPage = 10 #看要爬幾頁

In [3]:
def getTitle(board_soup):
    #get next N page
    for k in range(0,nPage):
        print('page:'+str(k)+'/'+str(nPage),end='\r')
        #first page url
        if k!=0:
            if len(board_soup.select('.btn.wide'))==0: #沒有上一頁
                break
            nextPage = 'https://www.ptt.cc'+board_soup.select('.btn.wide')[1]['href']
            board_html = requests.post(nextPage,data=over18)
            board_soup = BeautifulSoup(board_html.content, 'html.parser')
        #get title and url
        for node in board_soup.select('.title a'):
            content_title = node.text
            #print(content_title,end='  ')
            content_url = 'https://www.ptt.cc'+node['href']
            content_html = requests.post(content_url,data=over18)
            content_soup = BeautifulSoup(content_html.content, 'html.parser')
            content_date = content_soup.select('.article-meta-value')[3].text.strip()
            #print(content_date)
            #print(content_url)
            content = content_soup.select('#main-content')[0].text.strip()
            #remove first line and comments
            content = content[content.find('\n'):content.find('--')]
            #print(content)
            #print('---------------------------------------------------------------------------')
    print('Finish       ')

In [4]:
def getBoard(url):
    board_html = requests.post(url,data=over18)
    board_soup = BeautifulSoup(board_html.content,'html.parser')
    if len(board_soup.select('.board-label'))==0: #不是樹葉節點的板
        if len(board_soup.select('.b-ent'))==0: #壞掉的版裡面沒東西
            board_path.pop()
            return
        for board_url in board_soup.select('.board'):
            board_name = board_url.select('.board-name')[0].text.strip()
            if board_name=='0ClassRoot': #這個就會回到分類看板
                continue
            board_path.append(board_name)
            board_url = 'https://www.ptt.cc'+board_url['href'].strip()
            getBoard(board_url)
        board_path.pop()
    else: #是樹葉節點的板 像八卦版那樣 上面有個"看板"
        for path in board_path:
            print(path,end='->')
        print(board_soup.select('.board')[0].text)
        print('https://www.ptt.cc'+board_soup.select('.board')[0]['href'])
        board_path.pop()
        getTitle(board_soup)

In [5]:
getBoard(website)

A_Group->OtherBBS->ChinaBBS->CNBBS->SMTH->看板 SMTH
https://www.ptt.cc/bbs/SMTH/index.html
Finish       
A_Group->OtherBBS->ChinaBBS->CNBBS->PKU->看板 PKU
https://www.ptt.cc/bbs/PKU/index.html
Finish       
A_Group->OtherBBS->ChinaBBS->CNBBS->dopai->看板 dopai
https://www.ptt.cc/bbs/dopai/index.html


KeyError: 'href'